In [1]:
import numpy as np
import pandas as pd
from multiprocess import Pool
import itertools
from itertools import product
from math import prod as mul

Варіант 5

S = (F, 8, E, 9, 7, 2, 0, D, C, 6, 1, 5, B, 4, 3, A)

In [2]:
s_block = [0xF, 0x8, 0xE, 0x9, 0x7, 0x2, 0x0, 0xD, 0xC, 0x6, 0x1, 0x5, 0xB, 0x4, 0x3, 0xA]
s_block_inverse = [0x6, 0xA, 0x5, 0xE, 0xD, 0xB, 0x9, 0x4, 0x1, 0x3, 0xF, 0xC, 0x8, 0x7, 0x2, 0x0]
ENC = True
NOT_ENC = False

class heys:
    
    def heys_round(self, n, key = 0):
        ct = n ^ key
        ct = self.substitute(ct , s_block)
        ct = self.mix_words(ct)
        return ct

    # r = (s_1(y_1), s_2(y_2), ... , s_n(y_n)) 
    def substitute(self, n, enc):
        s = s_block if enc == True else s_block_inverse
        r = 0
        for i in range(4):
            r |= s[(n >> (i * 4)) & 15] << (i * 4)
        return r
        
    # i-тий біт j-того фрагменту стає j-тим бітом i-того фрагменту.
    def mix_words(self, n):
        r = 0
        for j in range(4):
            for i in range(4):
                r |= (n >> (4 * j + i) & 1) << (4 * i + j)
        return r
              

Створюємо таблицю диференціалів

$$ f(x \circ a) \bullet (f(x))^{-1} = b $$
так як у нас лише операції $\oplus$, тож воно претворюється на 
$$ f(x \oplus a) \oplus f(x) = b $$

In [3]:
heys_obj = heys()
# таблиця диференціалів для перестановки шифру Хейса
diff_substitution = np.zeros((16, 16))

for alpha in range(16):
    for beta in range(16):
        for x in range(16):
            diff_substitution[alpha][beta] += heys_obj.substitute(x ^ alpha, ENC) == heys_obj.substitute(x, ENC) ^ beta

diff_substitution /= 16

pd.DataFrame(diff_substitution)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.0,0.000,0.000,0.000,0.125,0.125,0.000,0.250,0.000,0.125,0.125,0.000,0.000,0.125,0.000,0.125
2,0.0,0.250,0.000,0.125,0.000,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.125,0.125,0.125
3,0.0,0.125,0.125,0.000,0.000,0.000,0.250,0.250,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.000
4,0.0,0.000,0.250,0.000,0.125,0.000,0.000,0.125,0.125,0.000,0.125,0.000,0.000,0.000,0.125,0.125
5,0.0,0.000,0.000,0.125,0.000,0.000,0.125,0.000,0.125,0.125,0.000,0.125,0.000,0.250,0.000,0.125
6,0.0,0.125,0.000,0.000,0.000,0.125,0.000,0.000,0.000,0.125,0.125,0.125,0.125,0.000,0.000,0.250
7,0.0,0.000,0.125,0.000,0.000,0.250,0.125,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.250,0.000
8,0.0,0.000,0.000,0.250,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.250,0.000,0.125,0.125
9,0.0,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.125,0.250,0.125,0.125,0.000,0.000,0.125,0.000


In [4]:
PROBABILITY_THRESHOLD = 0.0008
local_diff_table = {}

# betas
def get_new_betas(alpha):
    # obtain raw alpha values from one big Alpha
    alpha_list = [(alpha >> (4 * i)) & 15 for i in range(4)]
    # obtain non-null betas from s block diff table
    non_null_betas = []
    for i, alpha in enumerate(alpha_list):
        # filter betas with non-zero entries
        betas = [beta for beta in range(16) if diff_substitution[alpha][beta] != 0]
        non_null_betas.append(betas)
    
    betas = {}
    # generate all possible combinations that can appear in resulted beta
    for (i, beta_list) in enumerate(product(*non_null_betas, repeat=1)):
        beta = sum([beta_list[i] << (i * 4) for i in range(4)])
        betas[beta] = mul([diff_substitution[alpha_list[i]][beta_list[i]] for i in range(4)])
        
    return betas
    

def differential_search(alpha):
    # shared between workers variable
    global local_diff_table
    
    # Г_{t-1}(alpha) (Г spells like 'g')
    g_prev = {alpha: 1}
    for i in range(1, 6):
        g_current = {}
        # misuse of 'beta' naming (by now we are iterating over last alphas right before calculation of probabilities)
        for beta, p_i in g_prev.items():
            
            if beta not in local_diff_table:
                local_diff_table[beta] = get_new_betas(beta)
            
            for gamma in local_diff_table[beta]:
                tmp_prob = p_i * local_diff_table[beta][gamma]
                if gamma not in g_current:
                    g_current[gamma] = tmp_prob
                else:
                    g_current[gamma] += tmp_prob
            
        # Г_{t}(alpha)
        g_new_filtered = {}
        #check "bounds" and write to new list mixed words (aka gammas are becoming new alphas)
        for gamma in g_current.keys():
            if g_current[gamma] > PROBABILITY_THRESHOLD: 
                g_new_filtered[heys_obj.mix_words(gamma)] = g_current[gamma]
           
        g_prev = g_new_filtered
    
    return (alpha, g_prev)

Ініціалізуємо pool воркерів
[1](https://superfastpython.com/multiprocessing-pool-python/#What_Are_Python_Processes) 
[2](https://superfastpython.com/multiprocessing-pool-shared-global-variables/)

In [5]:
%%time

def prepare_alphas():
    alfas = []
    for i in range(4):
        for j in range(1, 16):
            alfas.append(j << 4 * i)
    return alfas

def init_worker(shared_diff_table):
    global local_diff_table
    local_diff_table = shared_diff_table

with Pool(initializer = init_worker, initargs = (local_diff_table,)) as pool:
    candidates = list(
        pool.map(
            differential_search,
            prepare_alphas()
        )
    )

print('Possible candidates after differential search: ')
list(candidates)

Possible candidates after differential search: 
CPU times: user 286 ms, sys: 39.3 ms, total: 325 ms
Wall time: 4.17 s


[(1, {}),
 (2, {}),
 (3, {}),
 (4, {546: 0.0009160041809082031}),
 (5, {}),
 (6, {}),
 (7, {}),
 (8, {}),
 (9, {}),
 (10, {}),
 (11, {}),
 (12, {}),
 (13, {}),
 (14, {}),
 (15, {}),
 (16, {}),
 (32, {}),
 (48, {}),
 (64, {1092: 0.000835418701171875}),
 (80, {}),
 (96, {}),
 (112, {}),
 (128, {}),
 (144, {}),
 (160, {}),
 (176, {}),
 (192, {}),
 (208, {}),
 (224, {}),
 (240, {}),
 (256, {}),
 (512, {}),
 (768, {273: 0.0009918212890625}),
 (1024,
  {273: 0.00206756591796875,
   4369: 0.00133514404296875,
   2184: 0.0014445781707763672,
   34952: 0.001047372817993164,
   17: 0.0010986328125,
   4352: 0.0008087158203125,
   4368: 0.00084686279296875,
   136: 0.0008516311645507812,
   546: 0.0009028911590576172}),
 (1280, {}),
 (1536, {}),
 (1792, {273: 0.0010528564453125}),
 (2048, {}),
 (2304, {}),
 (2560,
  {273: 0.001422882080078125,
   4369: 0.0009613037109375,
   2184: 0.0009412765502929688,
   17: 0.00081634521484375}),
 (2816,
  {273: 0.00080108642578125,
   4368: 0.0008010864257812

Витягуємо із можливих кандидатів список можливих ключів

In [6]:
def extract_keys():
    return 0